In [ ]:
# Removes lint errors from VS Code
from typing import Dict, TYPE_CHECKING, Tuple, List

if TYPE_CHECKING:
    import kedro

    catalog: kedro.io.data_catalog.DataCatalog
    session: kedro.framework.session.session.KedroSession
    pipelines: Dict[str, kedro.pipeline.pipeline.Pipeline]


In [ ]:
import os

VIEW = os.getenv("DATASET_VIEW") or "tab_adult"
TABLE = os.getenv("DATASET_TABLE") or "table"
MULTI_PROCESS = (
    os.getenv("MULTI_PROCESS") if os.getenv("MULTI_PROCESS") is not None else True
)

import numpy as np
import pandas as pd
from importlib import reload

bin: pd.DataFrame = catalog.load(f"{VIEW}.wrk.bin_{TABLE}")
random_state = catalog.load("params:random_state")


In [ ]:
bin.head()

,age_0,age_1,age_2,age_3,age_4,age_5,workclass_0,workclass_1,workclass_2,workclass_3,...,native-country_33,native-country_34,native-country_35,native-country_36,native-country_37,native-country_38,native-country_39,native-country_40,native-country_41,native-country_42
id,,,,,,,,,,,,,,,,,,,,,
4,False,True,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,True,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
9,True,True,True,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10,False,False,True,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
import pasteur.synth.privbayes as pb
reload(pb)

e = 1
beta = 0.1
e1 = beta * e
e2 = (1 - beta) * e

d = len(bin.keys())
n = len(bin)

noise_ratio = 4

k = pb.calc_k(d, n, e2, noise_ratio)

[08/04/22 14:57:52] INFO     Selected k=2 with usefulness= 5.09 >  4.00                             ]8;id=618906;file:///mnt/ext/projects/pasteur/src/pasteur/synth/privbayes.py\privbayes.py]8;;\:]8;id=101799;file:///mnt/ext/projects/pasteur/src/pasteur/synth/privbayes.py#37\37]8;;\

In [ ]:
assert all(dtype.name == "bool" for dtype in bin.dtypes)

In [ ]:
data = bin[["age_0", "workclass_0", "workclass_1", "workclass_3"]]

marginal = data.groupby(list(data.keys())).size()
marginal

age_0  workclass_0  workclass_1  workclass_3
False  False        False        False          1227
                                 True            193
                    True         False           483
       True         False        False          4563
True   False        False        False          1261
                                 True            211
                    True         False           538
       True         False        False          4548
dtype: int64

In [ ]:
from scipy.stats import laplace

noise_scale = 2*(d - k) / e2
noise = laplace.rvs(loc=0, scale=noise_scale, size=marginal.shape)
noise

array([ -60.28357558,  187.92522185, -450.38759065,  261.66961542,
        301.08972393,   38.81723785, -799.25659474, -316.26261872])

In [ ]:
noisy_marginal = marginal + noise
noisy_marginal = noisy_marginal.clip(0)
noisy_marginal = noisy_marginal / noisy_marginal.sum()
noisy_marginal, noisy_marginal.sum()

(
    age_0  workclass_0  workclass_1  workclass_3
False  False        False        False          0.093723
                                 True           0.030600
                    True         False          0.002620
       True         False        False          0.387568
True   False        False        False          0.125483
                                 True           0.020068
                    True         False          0.000000
       True         False        False          0.339938
dtype: float64,
    1.0
)

In [ ]:
noise_scale

320.0

In [ ]:
len(bin) / len(marginal)

1628.0

In [ ]:
def calc_noisy_marginal(data: pd.DataFrame):
    marginal = data.groupby(list(data.keys())).size()
    noise_scale = 2*(d - k) / e2
    noise = laplace.rvs(loc=0, scale=noise_scale, size=marginal.shape)

In [ ]:
n, d = bin.shape
n, d

(13024, 146)

In [ ]:
x = "age_0"
p = ["workclass_0", "workclass_1", "workclass_3"]

joint = bin.groupby([x] + p).size()
joint

age_0  workclass_0  workclass_1  workclass_3
False  False        False        False          1227
                                 True            193
                    True         False           483
       True         False        False          4563
True   False        False        False          1261
                                 True            211
                    True         False           538
       True         False        False          4548
dtype: int64

In [ ]:
x_marginal = joint.groupby(x).sum()

x_marginal

age_0
False    6466
True     6558
dtype: int64

In [ ]:
p_marginal = joint.groupby(p).sum()

p_marginal

workclass_0  workclass_1  workclass_3
False        False        False          2488
                          True            404
             True         False          1021
True         False        False          9111
dtype: int64

In [ ]:
contigency = pd.pivot(pd.DataFrame(bin.groupby([x] + p).size()).reset_index(), p, x).to_numpy(dtype="float")
contigency /= contigency.sum()
contigency

array([[0.09421069, 0.09682125],
       [0.0148188 , 0.01620086],
       [0.03708538, 0.04130835],
       [0.35035319, 0.34920147]])

In [ ]:
x_marginal = contigency.sum(axis=0)
x_marginal

array([0.49646806, 0.50353194])

In [ ]:
p_marginal = contigency.sum(axis=1)
p_marginal

array([0.19103194, 0.03101966, 0.07839373, 0.69955467])

In [ ]:
(contigency*np.log2(contigency/np.outer(p_marginal, x_marginal))).sum()

0.00019971752759421221

In [ ]:
%load_ext line_profiler

In [ ]:
par = [bin[pi] for pi in p]
chld = bin[x]
%timeit pd.crosstab(par, chld)

4.97 ms ± 77.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit 
joint_dist = bin.groupby([x] + p).size()
contigency_pd = pd.DataFrame(joint_dist).reset_index()
contigency_pd = pd.pivot(contigency_pd, p, x)

4.01 ms ± 7.78 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit joint_dist = bin.groupby([x] + p).size()


1.28 ms ± 1.55 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
def calc_mutual_information(data: pd.DataFrame, x: str, p: list[str]):
    joint_dist = data.groupby([x] + p).size()
    contigency_pd = pd.DataFrame(joint_dist).reset_index()
    contigency_pd = pd.pivot(contigency_pd, p, x)

    contigency = contigency_pd.to_numpy(dtype="float")
    cg = contigency / contigency.sum()

    # Marginals
    x_mar = contigency.sum(axis=0)
    x_mar /= x_mar.sum()
    p_mar = contigency.sum(axis=1)
    p_mar /= p_mar.sum()

    return np.sum(cg*np.log2(cg/np.outer(p_mar, x_mar)))


%lprun -f calc_mutual_information calc_mutual_information(bin, x, p)
# calc_mutual_information(bin, x, p)

Timer unit: 1e-06 s

Total time: 0.010258 s
File: /tmp/ipykernel_294820/872837184.py
Function: calc_mutual_information at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def calc_mutual_information(data: pd.DataFrame, x: str, p: list[str]):
     2         1       3669.0   3669.0     35.8      joint_dist = data.groupby([x] + p).size()
     3         1       2383.0   2383.0     23.2      contigency_pd = pd.DataFrame(joint_dist).reset_index()
     4         1       4109.0   4109.0     40.1      contigency_pd = pd.pivot(contigency_pd, p, x)
     5                                           
     6         1         29.0     29.0      0.3      contigency = contigency_pd.to_numpy(dtype="float")
     7         1         13.0     13.0      0.1      cg = contigency / contigency.sum()
     8                                           
     9                                               # Marginals
    10         1          5.

In [ ]:
import random
from itertools import combinations as n_choose_k

random.seed(0)

def greedy_bayes(data: pd.DataFrame, k: int):
    total_marginal = 0
    N = {}
    V = set()
    A = set(data.keys())
    d = len(data.keys())

    # Add root randomly
    x = random.sample(A, k=1)[0]
    N[x] = []
    V.add(x)

    for _ in range(d - 1):
        O = []

        for x in A - V:
            if len(V) > k:
                O += [(x, c) for c in n_choose_k(V, k)]
            else:
                O += [(x, V.copy())]

        x, p = O[0]
        N[x] = p
        V.add(x)
        print(f"{len(V)}: {len(O)}")
        total_marginal += len(O)

    print(total_marginal)
    return N, V

greedy_bayes(bin, 2);

2: 145
3: 144
4: 429
5: 852
6: 1410
7: 2100
8: 2919
9: 3864
10: 4932
11: 6120
12: 7425
13: 8844
14: 10374
15: 12012
16: 13755
17: 15600
18: 17544
19: 19584
20: 21717
21: 23940
22: 26250
23: 28644
24: 31119
25: 33672
26: 36300
27: 39000
28: 41769
29: 44604
30: 47502
31: 50460
32: 53475
33: 56544
34: 59664
35: 62832
36: 66045
37: 69300
38: 72594
39: 75924
40: 79287
41: 82680
42: 86100
43: 89544
44: 93009
45: 96492
46: 99990
47: 103500
48: 107019
49: 110544
50: 114072
51: 117600
52: 121125
53: 124644
54: 128154
55: 131652
56: 135135
57: 138600
58: 142044
59: 145464
60: 148857
61: 152220
62: 155550
63: 158844
64: 162099
65: 165312
66: 168480
67: 171600
68: 174669
69: 177684
70: 180642
71: 183540
72: 186375
73: 189144
74: 191844
75: 194472
76: 197025
77: 199500
78: 201894
79: 204204
80: 206427
81: 208560
82: 210600
83: 212544
84: 214389
85: 216132
86: 217770
87: 219300
88: 220719
89: 222024
90: 223212
91: 224280
92: 225225
93: 226044
94: 226734
95: 227292
96: 227715
97: 228000
98: 228144
99

In [ ]:
set([1,2,3]) - set([1,2,3])

set()

In [ ]:
joint_dist = bin.groupby([x] + p).size()
joint_dist

age_0  workclass_0  workclass_1  workclass_3
False  False        False        False          1227
                                 True            193
                    True         False           483
       True         False        False          4563
True   False        False        False          1261
                                 True            211
                    True         False           538
       True         False        False          4548
dtype: int64

In [ ]:
joint_dist.xs(False, level="workclass_3").array

<PandasArray>
[1227, 483, 4563, 1261, 538, 4548]
Length: 6, dtype: int64

In [ ]:
%%timeit
data = bin[[x] + p].to_numpy()
domain = data.max(axis=0) + 1

margin, _ = np.histogramdd(data, domain)
margin_sum = margin.sum()

x_idx = 0
p_idx = tuple(range(1, len(p) + 1))

j_mar = margin / margin_sum
x_mar = np.sum(margin, axis=p_idx) / margin_sum
p_mar = np.sum(margin, axis=x_idx) / margin_sum

1.46 ms ± 4.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
margin.sum(axis=tuple(range(len(p))))

array([12620.,   404.])

In [ ]:
x_mar = contigency.sum(axis=0)
x_mar /= x_mar.sum()
p_mar = contigency.sum(axis=1)
p_mar /= p_mar.sum()

return np.sum(cg*np.log2(cg/np.outer(p_mar, x_mar)))

85.2 ns ± 0.231 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
